In [ ]:
from random import randint
import numpy as np

import pandas as pd

from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv("https://www.dropbox.com/s/vz1l18qa9xxvhy3/demand_trend.csv?dl=1", index_col=0)

df['date'] = pd.to_datetime(df['date'])
q = df[ ['date', 'demand'] ]
q.plot(x='date', y='demand', figsize=(10,10))

In [ ]:
q

In [ ]:
df =  df.fillna( method='ffill' ).fillna( method='bfill' )
q = df #a copy for safe-keeping

In [ ]:
#a neat hack to create a simple increaseing col
df = df.reset_index()

In [ ]:
df = df.rename(columns={'index' : 'sic'})
df

In [ ]:
X = df[['sic']].values

In [ ]:
X

In [ ]:
y = df['demand']

In [ ]:
reg = LinearRegression().fit(X, y)

In [ ]:
pred_rng = [ [x] for x in range(1066, 2000) ] # 

In [ ]:
preds = reg.predict(pred_rng)

In [ ]:
end_date = pd.Timestamp("2021-12-01") + pd.Timedelta(days=933)

preds_daterange = pd.date_range( pd.Timestamp("2021-12-01"), end_date)

In [ ]:
preds_df = pd.DataFrame( columns=["demand_pred", "date"] )

In [ ]:
preds_df

In [ ]:
preds_df['date'] = preds_daterange
preds_df['demand_pred'] = preds

In [ ]:
preds_df

In [ ]:
total_df = pd.concat( [q, preds_df], axis=0 )

In [ ]:
total_df

In [ ]:
total_df = total_df.set_index("date")

In [ ]:
#a very simple forecast
total_df.plot(figsize=(20,20))

In [ ]:
#adding features!
#let's first see if including holiday date improves the prediction


In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

cal = calendar()
holidays = cal.holidays(start=df['date'].min(), end=df['date'].max())

df['holiday'] = df['date'].isin(holidays)

In [ ]:
#we need numbers tho!
df

In [ ]:
df['holiday'] = df['holiday'].astype(float)

In [ ]:
#bingo
df

In [ ]:
X = df[['sic', 'holiday']].values
X

In [ ]:
y = df['demand'].values

In [ ]:
reg.fit(X, y)

In [ ]:
pred_rng_df = pd.DataFrame()

pred_rng = [ x for x in range(1066, 2000) ]
pred_rng_df['sic'] = pred_rng

end_date = pd.Timestamp("2021-12-01") + pd.Timedelta(days=933)
preds_daterange = pd.date_range( pd.Timestamp("2021-12-01"), end_date)
pred_rng_df['date'] = preds_daterange

cal = calendar()
holidays = cal.holidays(start=pred_rng_df['date'].min(), end=pred_rng_df['date'].max())

pred_rng_df['holiday'] = pred_rng_df['date'].isin(holidays).astype(float)

pred_rng_df

In [ ]:
X_pred = pred_rng_df[['sic', 'holiday']].values

preds = reg.predict(X_pred)

preds_df['date'] = preds_daterange
preds_df['demand_pred'] = preds

In [ ]:
total_df = pd.concat( [q, preds_df], axis=0 )

In [ ]:
total_df

In [ ]:
total_df.set_index('date').plot(figsize=(20,20))

In [ ]:
#homework 
#let's try adding day of week (one-hot-encoding)

df['day_of_week_0'] = df['date'].apply( lambda x : x.day_of_week == 0 ).astype(float)
df['day_of_week_1'] = df['date'].apply( lambda x : x.day_of_week == 1 ).astype(float)
df['day_of_week_2'] = df['date'].apply( lambda x : x.day_of_week == 2 ).astype(float)
df['day_of_week_3'] = df['date'].apply( lambda x : x.day_of_week == 3 ).astype(float)
df['day_of_week_4'] = df['date'].apply( lambda x : x.day_of_week == 4 ).astype(float)
df['day_of_week_5'] = df['date'].apply( lambda x : x.day_of_week == 5 ).astype(float)
df['day_of_week_6'] = df['date'].apply( lambda x : x.day_of_week == 6 ).astype(float)

df

In [ ]:
X = df[['sic', 'holiday', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'day_of_week_6']].values
X

In [ ]:
y = df['demand'].values

In [ ]:
reg.fit(X, y)

In [ ]:
pred_rng_df['day_of_week_0'] = pred_rng_df['date'].apply( lambda x : x.day_of_week == 0 ).astype(float)
pred_rng_df['day_of_week_1'] = pred_rng_df['date'].apply( lambda x : x.day_of_week == 1 ).astype(float)
pred_rng_df['day_of_week_2'] = pred_rng_df['date'].apply( lambda x : x.day_of_week == 2 ).astype(float)
pred_rng_df['day_of_week_3'] = pred_rng_df['date'].apply( lambda x : x.day_of_week == 3 ).astype(float)
pred_rng_df['day_of_week_4'] = pred_rng_df['date'].apply( lambda x : x.day_of_week == 4 ).astype(float)
pred_rng_df['day_of_week_5'] = pred_rng_df['date'].apply( lambda x : x.day_of_week == 5 ).astype(float)
pred_rng_df['day_of_week_6'] = pred_rng_df['date'].apply( lambda x : x.day_of_week == 6 ).astype(float)

pred_rng_df

In [ ]:
X_pred = pred_rng_df.drop('date', axis=1).values #just faster than selecting one by one

preds = reg.predict(X_pred)

preds_df['date'] = preds_daterange
preds_df['demand_pred'] = preds


In [ ]:
total_df = pd.concat( [q, preds_df], axis=0 )

In [ ]:
total_df.set_index("date").plot(figsize=(20,20))

In [ ]:
#accuracy

df['retro_pred'] = reg.predict(X)
df[['demand', 'retro_pred']].plot(figsize=(20,20))

In [ ]:
from sklearn.metrics import r2_score

r2_score(df['demand'], df['retro_pred'])

In [ ]:
r2_score( df['demand'], [df['demand'].mean()]*len(df) )

In [ ]:
r2_score( df['demand'], df['demand'] )

In [ ]:
#homework
#try to work out which large effect is still missing from the model, and add a good guess at the missing feature 
#clue, look at the three "unexplained" peaks, which date based function does that match ;)